In [67]:
import pandas as pd
import plotly.express as px
from omegaconf import OmegaConf
from pathlib import Path

In [17]:
results = pd.read_csv("~/Documents/ml_project-results.csv")
results.head(2)

,fit_time,score_time,test_score,fold,hydra_dir
0,0.001979,0.000450,0.86,1,/Users/dunnkers/git/hydra-demo/multirun/2022-0...
1,0.001664,0.000276,0.84,2,/Users/dunnkers/git/hydra-demo/multirun/2022-0...


We now retrieve the associated config. All hydra jobs are stored in a local directory, containing a `.hydra` folder. In that folder there's `config.yaml`, `hydra.yaml` and `overrides.yaml`. We can use the overrides to find the best experiment.

In [56]:
configs = []

for hydra_dir, _ in results.groupby("hydra_dir"):
    overrides_path = Path(hydra_dir) / ".hydra" / "overrides.yaml"
    overrides = OmegaConf.load(overrides_path)
    overrides_str = " ".join(overrides)
    configs.append({
        "hydra_dir": hydra_dir,
        "overrides": overrides_str
    })

configs = pd.DataFrame(configs)
data = results.merge(configs, on="hydra_dir")
data.head(2)

,fit_time,score_time,test_score,fold,hydra_dir,overrides
0,0.001979,0.000450,0.86,1,/Users/dunnkers/git/hydra-demo/multirun/2022-0...,hyperparameters.n_estimators=1 hyperparameters...
1,0.001664,0.000276,0.84,2,/Users/dunnkers/git/hydra-demo/multirun/2022-0...,hyperparameters.n_estimators=1 hyperparameters...


Experiments sorted by test score:

In [61]:
data.groupby("overrides").mean() \
    .sort_values("test_score", ascending=False)

,fit_time,score_time,test_score,fold
overrides,,,,
hyperparameters.n_estimators=61 hyperparameters.max_depth=10,0.068770,0.002806,0.974,3.0
hyperparameters.n_estimators=81 hyperparameters.max_depth=10,0.086078,0.003154,0.960,3.0
hyperparameters.n_estimators=51 hyperparameters.max_depth=10,0.059916,0.002199,0.949,3.0
hyperparameters.n_estimators=21 hyperparameters.max_depth=10,0.026303,0.001146,0.929,3.0
hyperparameters.n_estimators=91 hyperparameters.max_depth=10,0.110804,0.003891,0.927,3.0
hyperparameters.n_estimators=11 hyperparameters.max_depth=10,0.014089,0.000758,0.918,3.0
hyperparameters.n_estimators=71 hyperparameters.max_depth=10,0.084707,0.003008,0.918,3.0
hyperparameters.n_estimators=41 hyperparameters.max_depth=10,0.051156,0.001944,0.900,3.0
hyperparameters.n_estimators=31 hyperparameters.max_depth=10,0.038915,0.001529,0.886,3.0


In [68]:
px.line(data.groupby("overrides").mean(), y="test_score")